# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [6]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
# YOUR CODE HERE
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,47.86,46,0,3.44,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,60.80,85,0,13.96,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,73.99,69,0,5.68,clear sky
3,3,Dover,US,39.1582,-75.5244,58.14,84,100,20.71,moderate rain
4,4,Touros,BR,-5.1989,-35.4608,80.83,63,5,16.31,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [7]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature for your trip? "))
# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature for your trip? "))

What is the minimum temperature for your trip? 48
What is the maximum temperature for your trip? 78


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [8]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>min_temp) &
                                       (city_data_df['Max Temp']<max_temp)]
                                    
# Display sample data
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hermanus,ZA,-34.4187,19.2345,60.80,85,0,13.96,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,73.99,69,0,5.68,clear sky
3,3,Dover,US,39.1582,-75.5244,58.14,84,100,20.71,moderate rain
5,5,Chitral,PK,35.8462,71.7858,64.00,63,3,3.44,clear sky
8,8,Kulmbach,DE,50.1000,11.4500,53.83,93,100,6.40,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [9]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hermanus,ZA,-34.4187,19.2345,60.80,85,0,13.96,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,73.99,69,0,5.68,clear sky
3,3,Dover,US,39.1582,-75.5244,58.14,84,100,20.71,moderate rain
5,5,Chitral,PK,35.8462,71.7858,64.00,63,3,3.44,clear sky
8,8,Kulmbach,DE,50.1000,11.4500,53.83,93,100,6.40,overcast clouds
9,9,Kamariotissa,GR,40.4833,25.4667,73.76,54,0,18.54,clear sky
11,11,Kualakapuas,ID,-3.0091,114.3876,75.04,96,100,1.95,light rain
12,12,Avarua,CK,-21.2078,-159.7750,73.45,68,40,11.50,scattered clouds
15,15,East London,ZA,-33.0153,27.9116,67.86,88,11,15.77,few clouds
16,16,Castro,BR,-24.7911,-50.0119,71.51,61,75,4.43,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [10]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Hermanus,ZA,60.80,clear sky,-34.4187,19.2345
2,Rikitea,PF,73.99,clear sky,-23.1203,-134.9692
3,Dover,US,58.14,moderate rain,39.1582,-75.5244
5,Chitral,PK,64.00,clear sky,35.8462,71.7858
8,Kulmbach,DE,53.83,overcast clouds,50.1000,11.4500
9,Kamariotissa,GR,73.76,clear sky,40.4833,25.4667
11,Kualakapuas,ID,75.04,light rain,-3.0091,114.3876
12,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750
15,East London,ZA,67.86,few clouds,-33.0153,27.9116
16,Castro,BR,71.51,broken clouds,-24.7911,-50.0119


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [11]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,60.80,clear sky,-34.4187,19.2345,
2,Rikitea,PF,73.99,clear sky,-23.1203,-134.9692,
3,Dover,US,58.14,moderate rain,39.1582,-75.5244,
5,Chitral,PK,64.00,clear sky,35.8462,71.7858,
8,Kulmbach,DE,53.83,overcast clouds,50.1000,11.4500,
9,Kamariotissa,GR,73.76,clear sky,40.4833,25.4667,
11,Kualakapuas,ID,75.04,light rain,-3.0091,114.3876,
12,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,
15,East London,ZA,67.86,few clouds,-33.0153,27.9116,
16,Castro,BR,71.51,broken clouds,-24.7911,-50.0119,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [12]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [13]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,60.80,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Rikitea,PF,73.99,clear sky,-23.1203,-134.9692,People ThankYou
3,Dover,US,58.14,moderate rain,39.1582,-75.5244,Bally's Dover Casino Resort
5,Chitral,PK,64.00,clear sky,35.8462,71.7858,PTDC Motel Chitral
8,Kulmbach,DE,53.83,overcast clouds,50.1000,11.4500,Taste Hotel Kulmbach
9,Kamariotissa,GR,73.76,clear sky,40.4833,25.4667,Niki Beach
11,Kualakapuas,ID,75.04,light rain,-3.0091,114.3876,Guest House Izonk
12,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
15,East London,ZA,67.86,few clouds,-33.0153,27.9116,Tu Casa
16,Castro,BR,71.51,broken clouds,-24.7911,-50.0119,CHACARA BAILLY


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [15]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,60.80,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Rikitea,PF,73.99,clear sky,-23.1203,-134.9692,People ThankYou
3,Dover,US,58.14,moderate rain,39.1582,-75.5244,Bally's Dover Casino Resort
5,Chitral,PK,64.00,clear sky,35.8462,71.7858,PTDC Motel Chitral
8,Kulmbach,DE,53.83,overcast clouds,50.1000,11.4500,Taste Hotel Kulmbach
9,Kamariotissa,GR,73.76,clear sky,40.4833,25.4667,Niki Beach
11,Kualakapuas,ID,75.04,light rain,-3.0091,114.3876,Guest House Izonk
12,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
15,East London,ZA,67.86,few clouds,-33.0153,27.9116,Tu Casa
16,Castro,BR,71.51,broken clouds,-24.7911,-50.0119,CHACARA BAILLY


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [16]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [17]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [18]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Misty Waves Boutique Hotel</dd>\n<dt>City</dt><dd>Hermanus</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 60.8 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>clear sky and 73.99 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Bally's Dover Casino Resort</dd>\n<dt>City</dt><dd>Dover</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>moderate rain and 58.14 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>PTDC Motel Chitral</dd>\n<dt>City</dt><dd>Chitral</dd>\n<dt>Country</dt><dd>PK</dd>\n<dt>Current Weather</dt><dd>clear sky and 64.0 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Taste Hotel Kulmbach</dd>\n<dt>City</dt><dd>Kulmbach</dd>\n<dt>Country</dt><dd>DE</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 53.83 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Niki Beach</dd>\n<

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [19]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,-34.4187,19.2345
2,-23.1203,-134.9692
3,39.1582,-75.5244
5,35.8462,71.7858
8,50.1000,11.4500
9,40.4833,25.4667
11,-3.0091,114.3876
12,-21.2078,-159.7750
15,-33.0153,27.9116
16,-24.7911,-50.0119


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [21]:
# Add a marker layer for each city to the map. 
# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(40.0, 80.0), zoom_level=2)
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))